# Mapping health needs
Data from GBD 2010 study

In [1]:
setwd('/media/igna/Elements/HotelDieu/Cochrane')

GBD <- read.table("Mapping_Cancer/Tables/GBD_data_per_country_and_28_diseases_2005.txt")
names(GBD)


[1] "causelevel1"  "causelevel2"  "causelevel3"  "causelevel4"  "causelevel5" 
 [6] "country_name" "region"       "year"         "nm_mean"      "rt_mean"     
[11] "Disease"      "Sup_region"

In [2]:
dis <- levels(GBD$Dis)
reg <- levels(GBD$Sup_reg)

S <- tapply(GBD$nm_mean,paste(GBD$Sup_region,GBD$Disease,sep="&"),sum)
L <- strsplit(names(S),"&")

M <- matrix(NA,ncol=length(reg),nrow=length(dis))

for(i in 1:length(S)){
M[dis==L[[i]][2],reg==L[[i]][1]] <- S[i]
}

M <- data.frame(M)
names(M) <- reg
rownames(M) <- dis

M$Tot <- apply(M,1,function(x){sum(x[!is.na(x)])})

write.table(M,"Mapping_Cancer/Tables/GBD_data_per_region_and_28_diseases_2005.txt")

In [3]:
head(M)

,"Central Europe, Eastern Europe, and Central Asia",High-income,Latin America and Caribbean,North Africa and Middle East,South Asia,"Southeast Asia, East Asia and Oceania",Sub-Saharian Africa,Tot
Cardiovascular and circulatory diseases,53161407,43545779,15012902,19214033,56395260,81711420,18363310,287404109
Chronic respiratory diseases,5644608,13641980,5500905,4781865,38910999,31793866,12211132,112485355
Cirrhosis of the liver,4090337,3995636,2376422,1471645,7748645,7368063,3411973,30462721
Congenital anomalies,2079423,2296894,3617508,5091068,12842242,10039357,7288013,43254504
"Diabetes, urinary diseases and male infertility",5049825,12338425,6866404,4671200,18349709,21111320,7434597,75821480
"Diarrhea, lower respiratory infections, meningitis, and other common infectious diseases",7281616,6847691,10635120,10785476,130854936,37348849,125531848,329285537


In [4]:
#In the 28 groups of diseases, were already supressed Other infectious diseases and Other endocrine...
dim(M)
#Database with the total burden (all disease) per region
#We verify the share of the two residual categories among total burden and among diseases

GBT <- read.table("Mapping_Cancer/Tables/Nb_DALYs_per_region_and_disease_all_periods.txt")
names(GBT)
head(GBT)

[1] 28  8

[1] "Region"     "Tot"        "Chronic"    "Neoplasms"  "Infectious"
[6] "MNN"        "Oth_cat"    "No_GBD"     "year"

,Region,Tot,Chronic,Neoplasms,Infectious,MNN,Oth_cat,No_GBD,year
5,South Asia,697.80303841399,179.7802647762,22.858405613,189.87788439026,141.16963043613,155.3808302784,8.73602292,2005
7,Sub-Saharian Africa,596.582826844082,75.735228049432,9.55703834615,322.082215372734,105.96421235067,74.663669324096,8.580463401,2005
6,"Southeast Asia, East Asia and Oceania",529.490064010115,196.380486711291,65.73905751199,69.9214090804583,39.6395935996662,153.72809400671,4.0814231,2005
2,High-income,259.415724301473,139.11925831137,43.29828369423,9.13335771904556,5.137103610761,60.684831275766,2.0428896903,2005
1,"Central Europe, Eastern Europe, and Central Asia",177.132061289357,83.15488645187,19.507980928,13.1877235074411,6.157954269146,54.5541020329,0.5694141,2005
3,Latin America and Caribbean,135.961356566376,54.567157278174,9.69736505098,17.6332723599138,14.8850908284083,37.9190546961,1.2594163528,2005


In [5]:
1-sum(M$Tot)/(sum(GBT$Tot[GBT$year==2005])*1e6)

[1] 0.01059768

Ok, less than 1.1% of the global total burden in 2005

In [6]:
#Share of Other categories among diseases (no Injuries) and across regions
Oth_cat_gbd <- sum(GBT$Tot[GBT$year==2005])*1e6 - sum(M$Tot)
GBD_diseases <- sum(GBT$Tot[GBT$year==2005])*1e6 - sum(M$Tot[rownames(M)=="Injuries"])
Oth_cat_gbd/GBD_diseases

[1] 0.01185066

Ok, less than 1.2% of the burden of diseases (no Injuries)

In [7]:
#Across regions
GBT <- GBT[order(GBT$Region),]
Oth_cat_gbd <- GBT$Tot[GBT$year==2005]*1e6 - apply(M[-ncol(M)],2,sum)
GBD_diseases <- GBT$Tot[GBT$year==2005]*1e6 - M[rownames(M)=="Injuries",-ncol(M)]
100*Oth_cat_gbd/GBD_diseases

,"Central Europe, Eastern Europe, and Central Asia",High-income,Latin America and Caribbean,North Africa and Middle East,South Asia,"Southeast Asia, East Asia and Oceania",Sub-Saharian Africa
Injuries,0.3743995,0.8745466,1.0785740,1.3076612,1.4102916,0.8776721,1.5428734


Residual categories have maximised GBD in SSA <= 1.6% of total burden of diseases

In [8]:
#What is the part of injuries among the total burden
sum(M$Tot[rownames(M)=="Injuries"])/(sum(GBT$Tot[GBT$year==2005])*1e6)

[1] 0.1057308

Injuries = 10.6% of total burden

In [9]:
#Burden of injuries across regions
M[rownames(M)=="Injuries",-ncol(M)]/(GBT$Tot[GBT$year==2005]*1e6)

,"Central Europe, Eastern Europe, and Central Asia",High-income,Latin America and Caribbean,North Africa and Middle East,South Asia,"Southeast Asia, East Asia and Oceania",Sub-Saharian Africa
Injuries,0.14139041,0.09953742,0.14117654,0.10613045,0.11228823,0.12174321,0.06779870


Injuries maximum in CEE and LATAM = 14.1% Minimum in SubSaharian Africa = 6.8%

In [10]:
#We supress Injuries from the burden
M <- M[rownames(M)!="Injuries",]
dim(M)

[1] 27  8

In [13]:
#Burden per region
apply(M,2,sum)/1e6
apply(M,2,sum)/sum(M$Tot)

Central Europe, Eastern Europe, and Central Asia 
                                        151.5179 
                                     High-income 
                                        231.5513 
                     Latin America and Caribbean 
                                        115.5074 
                    North Africa and Middle East 
                                        102.2734 
                                      South Asia 
                                        610.7119 
           Southeast Asia, East Asia and Oceania 
                                        460.9468 
                             Sub-Saharian Africa 
                                        547.5548 
                                             Tot 
                                       2220.0635 
                                         Prop_bd 
                                          0.0001

Central Europe, Eastern Europe, and Central Asia 
                                    6.824934e-02 
                                     High-income 
                                    1.042994e-01 
                     Latin America and Caribbean 
                                    5.202887e-02 
                    North Africa and Middle East 
                                    4.606778e-02 
                                      South Asia 
                                    2.750876e-01 
           Southeast Asia, East Asia and Oceania 
                                    2.076278e-01 
                             Sub-Saharian Africa 
                                    2.466393e-01 
                                             Tot 
                                    1.000000e+00 
                                         Prop_bd 
                                    4.504376e-08

In [12]:
#Burden per disease
M$Prop_bd <- 100*M$Tot/(sum(M$Tot))
head(M[order(M$Tot,decreasing=TRUE),c(ncol(M)-1:0)])

,Tot,Prop_bd
"Diarrhea, lower respiratory infections, meningitis, and other common infectious diseases",3.292855e+08,1.483226e+01
Cardiovascular and circulatory diseases,2.874041e+08,1.294576e+01
Neonatal disorders,2.202881e+08,9.922602e+00
Neoplasms,1.768685e+08,7.966824e+00
Mental and behavioral disorders,1.719144e+08,7.743670e+00
Musculoskeletal disorders,1.515548e+08,6.826597e+00


In [15]:
tail(M[order(M$Tot,decreasing=TRUE),c(ncol(M)-1:0)])

,Tot,Prop_bd
Oral disorders,1.433652e+07,6.457706e-01
Sexually transmitted diseases excluding HIV,1.292219e+07,5.820641e-01
Hepatitis,1.255105e+07,5.653463e-01
Gynecological diseases,9.710285e+06,4.373877e-01
Sudden infant death syndrome,1.877956e+06,8.459021e-02
Leprosy,8.539553e+03,3.846535e-04


In [19]:
8.539553e+03/1e6

[1] 0.008539553

In [20]:
write.table(M,"Mapping_Cancer/Tables/GBD_data_per_region_and_27_diseases_2005.txt")